In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import sqlite3
import os
import numpy as np
from requests_html import HTMLSession
from requests_file import FileAdapter

session = HTMLSession()

session.mount('file://',FileAdapter())
pwd = os.getcwd().replace("\\","/")
html_obj = session.get(f'file:///{pwd}/台股公告 _ Anue鉅亨.html')
soup = BeautifulSoup(html_obj.text,"html.parser")

In [3]:
titletime = []
id = []
idBefore = []
content = []
  

# # 今日新闻
inputdate = '2022-09-16'


# # 日期============================
times = soup.select('._67tN')
for time in times:
  head,mid,tail = time.find('time')['datetime'].partition('T') # partition‘T'之後分為頭部，中部，尾部三個
  titletime.append(head)
# print(len(titletime))

# 內容===============================================================================================================
titles = soup.select('._1xc2')
# 1.去除特殊字符
for title in titles:
  head,mid,tail = title.find('h3').text.partition(':')
  if head[0] == '「':
    head2 = head.replace('「','') # replace（代替），將字符替換成
    head3 = head2.find('、') # 在第幾位

    if head3 < 0: # 沒有、號
      head2 = head2.split('」',1) # split（切割），切割為【」以前的內容】和【」以後的內容】
      head2 = head2[0]
      idBefore.append(head2)
    else: # 有、號
      head2 = head2[:head3]
      head2 = head2.split('」',1) 
      head2 = head2[0]
      idBefore.append(head2)
  else:
    idBefore.append(head)
    titleText = title.text
    titleText = titleText.strip()
  content.append(titleText)
# 2.去除【月營收】
for x in idBefore:
  x2 = x.find('月營收')
  if x2 < 0: # 沒有【月營收】
    id.append(x)
  else: # 有【月營收】
    x3 = x2 - 1
    x4 = x[:x3]
    id.append(x4)
# =================================================================================================================


data = {
  'date': titletime,
  'code': id,
  'content': content
}

df = pd.DataFrame(data)

# 去除特殊字符 ===================================================
for x in range(len(df['content'])):
  df.iloc[x]['content'] = df.iloc[x]['content'].replace('','')

# ===============================================================


newsToday = pd.Timestamp.today().strftime("%Y-%m-%d") # 格式化日期
df_mask = df['date']== inputdate            # 日期為今天的存檔
positions = np.flatnonzero(df_mask)
filtered_df=df.iloc[positions]
# print(filtered_df)


# connecting to sqlite
conn = sqlite3.connect('anuenews.db')
# cursor object
cursor = conn.cursor()
uniqueSql = '''select * from news t'''


for index, row in filtered_df.iterrows():
  try:
    cursor.execute(
        """INSERT OR IGNORE INTO news 
          (date, code, content)
          values(?,?,?)""",
          (row['date'],
          row['code'],
          row['content']),
        )
    conn.commit()
  except:
    pass

# print(filtered_df)

# 確認當日新聞 ========================================================================
dfToday = pd.read_sql(uniqueSql,conn)
# df_maskToday = dfToday['date']==newsToday            # 日期為今天的存檔
df_maskToday = dfToday['date']==inputdate           # 日期為今天的存檔
positionsToday = np.flatnonzero(df_maskToday)
filtered_dfToday=dfToday.iloc[positionsToday]
# ======================================================================================

filtered_dfToday

,date,code,content
4172,2022-09-16,宏全,宏全8月營收25.98億元年增39.33% 1—8月達171.47億元
4173,2022-09-16,慶豐富,慶豐富8月營收4.16億元年減2.94% 1—8月達36.50億元
4174,2022-09-16,台火,台火8月營收0.02億元年減93.17% 1—8月達0.70億元
4175,2022-09-16,欣泰,欣泰8月營收2.00億元年增8.08% 1—8月達17.32億元
4176,2022-09-16,盛弘,盛弘8月營收3.63億元年增21.72% 1—8月達28.55億元
4177,2022-09-16,日友,日友8月營收3.12億元年增7.95% 1—8月達25.65億元
4178,2022-09-16,陞泰,陞泰8月營收1.04億元年減7.57% 1—8月達8.57億元
4179,2022-09-16,宇智,宇智8月營收2.17億元年增77.21% 1—8月達16.72億元
4180,2022-09-16,統新,統新8月營收0.53億元年增48.96% 1—8月達4.58億元
4181,2022-09-16,立端,立端8月營收8.39億元年增45.98% 1—8月達57.85億元


In [1]:
# 创建资料库 ====================================================================
# import sqlite3


# # connecting to sqlite
# conn = sqlite3.connect('anuenews.db')
# # cursor object
# cursor = conn.cursor()

# # Creating table
# news = """ CREATE TABLE news (
#     date DATE,
#     code TEXT,
#     content TEXT UNIQUE, 
#     UNIQUE(content)
# ); """

# cursor.execute(news)
# ================================================================================